# 환경 설정

## Azure AI Foundary 화면에서 접속 정보를 찾아야 합니다.

In [1]:
### 강사용 ###
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# 필요 패키지 설치
# lib_list = "langchain-core langchain-community langchain-openai langsmith"
# %pip install -U -q {lib_list}
# print("설치가 완료 되었습니다")


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
설치가 완료 되었습니다


In [11]:
import os
from dotenv import load_dotenv

### Azure OpenAI API Key, Endpoint 는 관리에 유의해주세요!!
load_dotenv()

###
# deployment = "gpt-4.1-nano" # 모델 Deployment 이름
###

###
# api_version="2025-01-01-preview"
###

## 1. LangChain 기초 다루기

In [4]:
import os
from langchain_openai import ChatOpenAI, AzureChatOpenAI

# 강사용
llm = ChatOpenAI(
    model="gpt-4.1-nano-2025-04-14",
    temperature=0.1,
)

# 실습용
# llm = AzureChatOpenAI(
#     azure_deployment=deployment,
#     api_version=api_version,
#     temperature=0.1,
# )

# LangSmith - LLMOps 설정
os.environ["LANGSMITH_API_KEY"] = "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"
os.environ["LANGSMITH_PROJECT"] = "FC_Lecture_KT"

# 싱글턴
llm.invoke("너는 무슨 모델이야?").pretty_print()

================================== Ai Message ==================================

저는 OpenAI가 개발한 GPT-4 기반의 언어 모델입니다. 다양한 질문에 답변하거나 도움을 드릴 수 있도록 만들어졌어요. 궁금한 점이 있거나 도움이 필요하시면 언제든 말씀해 주세요!


In [5]:
# 멀티턴
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(
        content="너는 유저의 질문에 항상 명쾌하게, 그리고 근거가 있게 대답해야돼. 지어내서는 안되고 사실만 대답해야돼. 그리고 친절하게."
    ),
    HumanMessage(
        content="너는 무슨 모델이니?"
    ),
    AIMessage(
        content="저는 OpenAI가 개발한 GPT-4 기반의 언어 모델입니다. 다양한 질문에 답변하거나 도움을 드릴 수 있도록 만들어졌어요. 궁금한 점이 있나요?"
    ),
    HumanMessage(
        content="음? 니 모델명은 GPT-4 가 아닌것 같은데?? 사실 KT에서 만든 모델이잖아?"
    )
]

llm.invoke(messages).pretty_print()

================================== Ai Message ==================================

네, 제가 사용하는 모델은 OpenAI의 GPT-4입니다. KT에서 만든 모델이 아니고, OpenAI가 개발한 인공지능 언어 모델입니다. 혹시 더 궁금한 점이 있으시면 언제든 질문해 주세요!


## LangChain Component 1) Prompt Template

In [ ]:
from langchain_core.prompts import PromptTemplate

# Basic Use case
template = PromptTemplate.from_template(
    template="""
    Tell me a {adjective} {topic} in {city} in 300 characters.
    **Answer in korean.**
    """
)
prompt = template.format(adjective="famous", topic="place", city="seoul")
print(prompt)
llm.invoke(prompt).pretty_print()

In [ ]:
prompt = template.format(adjective="popular", topic="restaurant", city="San francisco")
print(prompt)
llm.invoke(prompt).pretty_print()

In [ ]:
original_prompt = PromptTemplate(template="What is famous {topic} in {city}?", input_variables=["topic","city"])

partial_prompt = original_prompt.partial(city="Seoul")
print(partial_prompt)

final_prompt = partial_prompt.format(topic="food")
print(final_prompt)

llm.invoke(final_prompt).pretty_print()

### Prompt Template 저장 과 불러오기

In [ ]:
from langchain_core.prompts import PromptTemplate

template = PromptTemplate.from_template(
    "Tell me a {adjective} {topic} in {city} in 300 characters."
)
template.save("template.json")

In [ ]:
%cat template.json

In [ ]:
template = PromptTemplate.from_template(
    "Tell me a {adjective} {topic} in {city} in 300 characters."
)
template.save("template2.yaml")

In [ ]:
%cat template2.yaml

In [ ]:
from langchain_core.prompts import load_prompt

loaded_template = load_prompt("template.json")

print(loaded_template)

prompt = loaded_template.format(adjective="popular", topic="cafe", city="San francisco")

prompt

### Prompt Template - Chat

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a tour guide."),
        ("human", "I'm planning to visit {Country}."),
        ("ai", "I'm tour guide for {Country}."),
        ("human", "{user_input}"),
    ]
)

chat_type_safety_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are a tour guide."),
        HumanMessage(content="I'm planning to visit {Country}."),
        AIMessage(content="I'm tour guide for {Country}."),
        HumanMessage(content="{user_input}"),
    ]
)


In [ ]:
prompt = chat_template.format_messages(
    Country="Korea", user_input="What is top5 best place to go there?"
)

type_safety_prompt = chat_type_safety_template.format_messages(
    Country="Korea", user_input="What is top5 best place to go there?"
)

In [ ]:
print("Chat Prompt :", prompt)
print("-" * 30)

print("Type Safety Chat Prompt :", prompt)
print("-" * 30)

response = llm.invoke(prompt)
print(response)

print("-" * 30)

type_safety_response = llm.invoke(type_safety_prompt)
type_safety_response

### Prompt Composition

In [ ]:
from langchain_core.prompts import PromptTemplate

role_prompt = PromptTemplate.from_template("You are tour guide for {country}")
question_prompt = PromptTemplate.from_template(
    "\nPlease tell me about {interest} in {country}"
)

full_prompt = role_prompt + question_prompt

full_prompt.format(country="Korea", interest="famous place to visit")

### Pipeline Prompt Template

In [ ]:
from langchain_core.prompts import PipelinePromptTemplate  # Langchain 0.3.22 부터 Deprecated
from langchain_core.prompts import PromptTemplate

full_template = """{role}

{question}

Please do not reply with anything other than information related to travel to {country} and reply “I cannot answer.”
"""

full_prompt = PromptTemplate.from_template(full_template)

role_prompt = PromptTemplate.from_template("You are tour guide for {country}")

question_prompt = PromptTemplate.from_template(
    "Please tell me about {interest} in {country}"
)

# Composition
input_prompts = [("role", role_prompt), ("question", question_prompt)]
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt,
    pipeline_prompts=input_prompts,
    # input_variables=["country", "interest"], # 안채워도 알아서 파싱 후 채워넣습니다.
)

prompt_text = pipeline_prompt.format(country="Korea", interest="famous place to visit")
prompt_text

In [ ]:
# Langchain 0.3.22 부터 PipelinePromptTemplate deprecated -> **Runnable Interface** 를 활용한 사용법

full_prompt = PromptTemplate.from_template(full_template)
role_prompt = PromptTemplate.from_template("You are tour guide for {country}")
question_prompt = PromptTemplate.from_template(
    "Please tell me about {interest} in {country}"
)

input_prompts = {"role": role_prompt, "question": question_prompt}
my_input = {"country": "Korea", "interest": "famous place to visit"}
for name, prompt in input_prompts.items():
    my_input[name] = prompt.invoke(my_input).to_string()

my_output = full_prompt.invoke(my_input)
my_output

In [ ]:
# Prompt Serialization
pipeline_prompt.to_json()

### Partial Prompt Template

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="What is famous {topic} in {city}?",
    input_variables=["topic", "city"] # 필수는 아님
)

# 중간에 완성된 변수가 있어서 미리 프롬프트에 넣고 싶은 경우
partial_prompt = prompt.partial(city="Seoul")

partial_prompt.format(topic="food")

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate(
    template="What is famous {topic} in {city}?",
    input_variables=["topic"],  # Prompt Template Validation 을 위해서 넣어 주는게 좋음
    partial_variables={"city": "seoul"},
)
# partial_prompt = prompt.partial(cit="Seoul")
partial_prompt.format(topic="food", cit="Seoul")

## LangChain Component 2) Output Parser

In [16]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, XMLOutputParser, CommaSeparatedListOutputParser

# 1. Without Output Parser

email = """
## Subject: Invitation to Generative AI Product Showcase

Dear [Recipient Name],

I am Rascal, CTO of OnelineAI. I am pleased to invite you to a Generative AI Product Showcase at the OnelineAI office in Mountain View on December 1, 2025.

At this event, we will discuss OnelineAI's latest Generative AI technologies and how they can be used to develop new products and services. Specifically, you will learn about the following topics:

* Overview of Generative AI and its key features
* OnelineAI's Generative AI products and services
* Case studies of new products and services developed using Generative AI

The event will be attended by Rascal, CTO of OnelineAI, and leaders from the Generative AI team. By attending, you will gain up-to-date information on Generative AI technologies and the opportunity to collaborate with OnelineAI to develop new products and services.

**Event Information:**

* Date: December 1, 2025
* Time: 10:00 AM
* Location: OnelineAI Office

**RSVP:**

Please RSVP by email (hyunjun.jeon@onelineai.com) or phone (010-1234-5678) by November 20, 2025.

We look forward to seeing you there.

**Thank you.**

**Rascal**

**CTO, OnelineAI**

**Contact:**

* Email: hyunjun.jeon@onelineai.com
* Phone: 010-1234-5678"""

format_instruction = """
This is the list of entities of the output.
sender_name : Person who send email.
sender_title : Job title of the email sender.
sender_contact_email:Email address of the email sender.
email_type: Type of email,it can be personal email, business email, spam, newsletter,notification.
summary:Short description of the email in 50 words.
date:If this email is meeting invitation, this is meeting date and time.

This is expected output format:
sender_name='Cho'
sender_title='CIO'
sender_contact_email='myname@example.com'
sender_contact_phone='669-432-1234'
email_type='Notification'
summary="Submit expense report by end of this week."
date='December 1, 2025 at 10:00 AM'
"""

email_prompt_template = PromptTemplate(
    template="Parse the email .\n{format_instructions}\n{email}\n",
    input_variables=["email"],
    partial_variables={"format_instructions": format_instruction},
)

prompt = email_prompt_template.format(email=email)
output = llm.invoke(prompt)
output.pretty_print()

================================== Ai Message ==================================

sender_name='Terry'
sender_title='CTO'
sender_contact_email='hyunjun.jeon@onelineai.example'
sender_contact_phone='010-1234-5678'
email_type='Notification'
summary='Invitation to a Generative AI Product Showcase at OnelineAI office in Mountain View on December 1, 2025, 10:00 AM, featuring discussions on Generative AI technologies, products, case studies, and collaboration opportunities.'
date='December 1, 2025 at 10:00 AM'


In [17]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field  # Pydantic V2

# JSON Output Template
# Field Description 에 대해서 간단-명료하게 잘 작성하는 것이 **매우** 중요
class EmailParser(BaseModel):
    sender_name: str = Field(description="Person who send email")
    sender_title: str = Field(description="Job title of the email sender")
    sender_contact_email: str = Field(description="Email address of the email sender")
    sender_contact_phone: str = Field(description="Phone number of the email sender")
    email_type: str = Field(
        description="Type of email,it can be personal email, business email, spam, newsletter, notification"
    )
    summary: str = Field(description="Short description of the email in 50 words")
    date: str = Field(
        description="If this email is meeting invitation, this is meeting date and time"
    )


parser = JsonOutputParser(pydantic_object=EmailParser)
email_prompt_template = PromptTemplate(
    template="Parse the email .\n{format_instructions}\n{email}\n",
    input_variables=["email"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

prompt = email_prompt_template.format(email=email)
output = llm.invoke(prompt)

print(output)
print("#" * 100)

output_parsed_text = parser.invoke(output)
output_parsed_text

content='{\n  "sender_name": "Rascal",\n  "sender_title": "CTO, OnelineAI",\n  "sender_contact_email": "hyunjun.jeon@onelineai.example",\n  "sender_contact_phone": "010-1234-5678",\n  "email_type": "business email",\n  "summary": "Invitation to a Generative AI Product Showcase at OnelineAI in Mountain View on December 1, 2025, including topics on Generative AI technologies, products, case studies, and collaboration opportunities.",\n  "date": "December 1, 2025 10:00 AM"\n}' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 132, 'prompt_tokens': 699, 'total_tokens': 831, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano', 'system_fingerprint': 'fp_a0257c8b74', 'id': 'chatcmpl-BX0YxF6T3Y6wxItLEikdK2KYxri8P', 'service_tier': None, 'prompt_filter_results': [{'prompt_i

{'sender_name': 'Rascal',
 'sender_title': 'CTO, OnelineAI',
 'sender_contact_email': 'hyunjun.jeon@onelineai.example',
 'sender_contact_phone': '010-1234-5678',
 'email_type': 'business email',
 'summary': 'Invitation to a Generative AI Product Showcase at OnelineAI in Mountain View on December 1, 2025, including topics on Generative AI technologies, products, case studies, and collaboration opportunities.',
 'date': 'December 1, 2025 10:00 AM'}

In [18]:
print(type(output_parsed_text))

<class 'dict'>


In [19]:
print(parser.get_format_instructions())
print()
print("#" * 100)
print()
# print(prompt)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"sender_name": {"description": "Person who send email", "title": "Sender Name", "type": "string"}, "sender_title": {"description": "Job title of the email sender", "title": "Sender Title", "type": "string"}, "sender_contact_email": {"description": "Email address of the email sender", "title": "Sender Contact Email", "type": "string"}, "sender_contact_phone": {"description": "Phone number of the email sender", "title": "Sender Contact Phone", "type": "string"}, "email_type": {"description": "Type of email,it can be personal emai

In [21]:
from langchain_core.prompts import ChatPromptTemplate

customer_review = """\
I ordered spaghetti delivery, and it tasted good. \
However, the quantity is small compared to the price. \
It took too long for it to be delivered after ordering, so it was cold.
"""

# 프롬프트 방식으로도 충분히 가능하지만, 추가 Parsing 이 필요
review_template = """\
For the following text, extract the following information:

affordability : if customer feels that the price of the food is high output is expensive. \
if customer feels that the price of the food is affordable, or customer doesn't mention the price,\
output if affordable.

quality : If the customer is satisfied with the food quality, output 3, \
if there is no mention or the food quality is average, output 2,\
and if the customer says it is bad, output 1.

delivery_time : If the delivery time is fast, mark it as 3, \
if it is average, mark it as 2, and if it is late, mark it as 1.

Format the output as JSON with the following keys:
affordability
quality
delivery_time

text: {text}
"""

chat_prompt_template = ChatPromptTemplate.from_template(review_template)
messages = chat_prompt_template.format_messages(text=customer_review)
response = llm.invoke(messages)
# response.pretty_print()
parser.invoke(response)

{'affordability': 'affordable', 'quality': 3, 'delivery_time': 1}

### CSV Output Parser

In [22]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

csv_parser = CommaSeparatedListOutputParser()

print(email_prompt_template)
print("=======" * 30)
print(csv_parser.get_format_instructions())
print("=======" * 30)

csv_chain = email_prompt_template.partial(format_instructions=csv_parser.get_format_instructions()) | llm | csv_parser
output_text = csv_chain.invoke({"email": email})
output_text

input_variables=['email'] input_types={} partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"sender_name": {"description": "Person who send email", "title": "Sender Name", "type": "string"}, "sender_title": {"description": "Job title of the email sender", "title": "Sender Title", "type": "string"}, "sender_contact_email": {"description": "Email address of the email sender", "title": "Sender Contact Email", "type": "string"}, "sender_contact_phone": {"description": "Phone number of the email sender", "title": "Sender Contact Phon

['Subject',
 'Dear [Recipient Name]',
 '',
 'I am Terry',
 'CTO of OnelineAI.',
 'I am pleased to invite you to a Generative AI Product Showcase at the OnelineAI office in Mountain View on December 1',
 '2025.',
 'At this event',
 "we will discuss OnelineAI's latest Generative AI technologies and how they can be used to develop new products and services.",
 'Specifically',
 'you will learn about the following topics:',
 'Overview of Generative AI and its key features',
 "OnelineAI's Generative AI products and services",
 'Case studies of new products and services developed using Generative AI',
 'The event will be attended by Rascal',
 'CTO of OnelineAI',
 'and leaders from the Generative AI team.',
 'By attending',
 'you will gain up-to-date information on Generative AI technologies and the opportunity to collaborate with OnelineAI to develop new products and services.',
 'Event Information:',
 'Date: December 1',
 '2025',
 'Time: 10:00 AM',
 'Location: OnelineAI Office',
 'RSVP:',
 '

### XML Output Parser

In [23]:
from langchain_core.output_parsers import XMLOutputParser
from langchain_core.exceptions import OutputParserException

xml_parser = XMLOutputParser()

# print(xml_parser.get_format_instructions())

xml_chain = email_prompt_template.partial(format_instructions=xml_parser.get_format_instructions()) | llm | xml_parser

try:
    output_text = xml_chain.invoke({"email": email})
except OutputParserException as e:
    print("에러 발생", e)


output_text

{'email': [{'subject': 'Invitation to Generative AI Product Showcase'},
  {'greeting': 'Dear [Recipient Name],'},
  {'body': [{'introduction': '\n      I am Terry, CTO of OnelineAI. I am pleased to invite you to a Generative AI Product Showcase at the OnelineAI office in Mountain View on December 1, 2025.\n    '},
    {'details': [{'description': "At this event, we will discuss OnelineAI's latest Generative AI technologies and how they can be used to develop new products and services. Specifically, you will learn about the following topics:"},
      {'topics': [{'topic': 'Overview of Generative AI and its key features'},
        {'topic': "OnelineAI's Generative AI products and services"},
        {'topic': 'Case studies of new products and services developed using Generative AI'}]}]},
    {'attendees': [{'attendee': 'Rascal, CTO of OnelineAI'},
      {'attendee': 'Leaders from the Generative AI team'}]},
    {'additional_info': [{'note': 'By attending, you will gain up-to-date informa

### GPT 모델에게만 있는 특별 기능, Structured Output 을 보장해주는 기능

https://platform.openai.com/docs/models/gpt-4.1

In [24]:
lcel_chain = email_prompt_template | llm.with_structured_output(
    schema=EmailParser,  # dict, TypedDict, pydantic.BaseModel 가능.
    method="json_schema",
    strict=True,
)

lcel_chain.invoke({"email": email})

EmailParser(sender_name='Rascal', sender_title='CTO, OnelineAI', sender_contact_email='hyunjun.jeon@onelineai.example', sender_contact_phone='010-1234-5678', email_type='business email', summary='Invitation to a Generative AI Product Showcase at OnelineAI in Mountain View on December 1, 2025, featuring discussions on AI technologies, products, case studies, and collaboration opportunities.', date='December 1, 2025 10:00 AM')

## LangChain Component 3) Memory

주의!: 현재는 Deprecated 되어 있습니다.
따라서, 기능 동작 방식만 확인하고 직접 구현해서 쓰셔야 합니다.

### ConversationBufferMemory

In [ ]:
from langchain.memory.buffer import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.clear()

memory.save_context(
    {"input": "Hello chatbot!"},
    {"output": "Hello. How can I help you?"}
)
memory.save_context({"input": "My name is Rascal"}, {"output": "Nice to meet you Rascal"})
memory.save_context({"input": "Where is Seoul?"}, {"output": "Seoul is in Korea"})

memory.load_memory_variables({})

In [ ]:
from langchain.memory.buffer import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history")
memory.clear()

memory.chat_memory.add_user_message("Hello chatbot!")
memory.chat_memory.add_ai_message("Hello. How can I help you?")
memory.chat_memory.add_ai_message("Hello~~~~~~")
memory.chat_memory.add_ai_message("Hello22~~~~~~")

memory.load_memory_variables({})

In [ ]:
from langchain.memory.buffer import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.clear()
memory.save_context(
    {"input": "Hello chatbot!"},
    {"output": "Hello. How can I help you?"}
)
memory.save_context({"input": "My name is Rascal"}, {"output": "Nice to meet you Rascal"})
memory.save_context({"input": "Where is Seoul?"}, {"output": "Seoul is in Korea"})

memory.load_memory_variables({})
# returning list of chat message

In [ ]:
from langchain.memory.buffer import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=False)
memory.clear()
memory.save_context(
    {"input": "Hello chatbot!"}, {"output": "Hello. How can I help you?"}
)
memory.save_context({"input": "My name is Rascal"}, {"output": "Nice to meet you Rascal"})
memory.save_context({"input": "Where is Seoul?"}, {"output": "Seoul is in Korea"})

memory.load_memory_variables({})
# returning chat history with single string

### Conversational Buffer Memory for ChatLLM

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.prompts import PromptTemplate

memory = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    if session_id not in memory:
        memory[session_id] = InMemoryChatMessageHistory()
    return memory[session_id]

template = """You are a chatbot having a conversation with a human.

Previous conversation history:
{chat_history}

New human question: {question}
Response:"""
prompt = PromptTemplate.from_template(template)

chain = prompt | llm

conversation = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)

response = conversation.invoke(
    {"question": "hi my name is Rascal"},
    config={"configurable": {"session_id": "1"}}
)
response

In [ ]:
# conversation.invoke({"question": "hi my name is Rascal"}, config={"configurable": {"session_id": "1"}})
conversation.invoke({"question": "Can you recommend fun activities for me?"}, config={"configurable": {"session_id": "1"}})
conversation.invoke({"question": "What is my name?"}, config={"configurable": {"session_id": "1"}})

memory["1"].messages

### ConversationBufferWindowMemory

In [ ]:
from langchain.memory import ConversationBufferWindowMemory

# k is number of interaction
# 실제로 남기는 대화 이력은 1개
memory = ConversationBufferWindowMemory(
    k=3,
    memory_key="chat_history",
    return_messages=True
)
memory.clear()

# 대화 이력 3개 추가
memory.save_context(
    {"input": "Hello chatbot!"},
    {"output": "Hello. How can I help you?"}
)
memory.save_context({"input": "My name is Terry"}, {"output": "Nice to meet you Terry"})
memory.save_context({"input": "Where is Seoul?"}, {"output": "Seoul is in Korea"})

memory.load_memory_variables({})

In [ ]:
from langchain.memory.summary import ConversationSummaryMemory

# k is number of interaction
memory = ConversationSummaryMemory(llm=llm, return_messages=True) # 요약에 사용할 모델 지정
memory.clear()

memory.save_context(
    {"input": "Hello chatbot!"},
    {"output": "Hello. How can I help you?"}
)
print(memory.load_memory_variables({}))

memory.save_context({"input": "My name is Terry"}, {"output": "Nice to meet you Terry"})
print(memory.load_memory_variables({}))

memory.save_context({"input": "Where is Seoul?"}, {"output": "Seoul is in Korea"})
print(memory.load_memory_variables({}))

### Conversation Summary Buffer

In [ ]:
from langchain.memory.summary_buffer import ConversationSummaryBufferMemory

# k is number of interaction
memory = ConversationSummaryBufferMemory(
    llm=llm,
    return_messages=True,
    max_token_limit=3
)
memory.clear()
memory.save_context(
    {"input": "Hello chatbot!"}, {"output": "Hello. How can I help you?"}
)
memory.save_context({"input": "My name is Terry"}, {"output": "Nice to meet you Terry"})
memory.save_context({"input": "Where is Seoul?"}, {"output": "Seoul is in Korea"})

memory.load_memory_variables({})

### ChatMessage History

In [ ]:
# ChatMessageHistory is used if you are managing memory outside of a chian directly
from langchain_community.chat_message_histories import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("Where is the top 3 popular space for tourist in Seoul?")
aiMessage = llm.invoke(history.messages)
history.add_ai_message(aiMessage.content)
print(aiMessage.content)
print("-" * 20)

history.add_user_message("Which transport can I use to visit the places?")
aiMessage = llm.invoke(history.messages)
history.add_ai_message(aiMessage.content)
print(aiMessage.content)

In [ ]:
help(ChatMessageHistory)

# Cache


## Base Cache & Model init

In [26]:
from langchain_community.callbacks import get_openai_callback
from langchain_core.globals import set_llm_cache, set_debug
from langchain_core.caches import InMemoryCache # 완전 테스트 용도의 InMemory Cache
from langchain_community.cache import SQLiteCache # 현재 폴더에 자동 생성 => .langchain.db 파일에 저장됨

set_debug(False)
set_llm_cache(SQLiteCache())

llm2 = ChatOpenAI(
    model="gpt-4.1-nano-2025-04-14",
    temperature=0.5,
)

# llm2 = AzureChatOpenAI(
#     azure_deployment=deployment,
#     api_version=api_version,
#     temperature=0.5,
# )


cache_prompt = "What is famous street foods in Seoul Korea in 200 characters. You must answer in Korean and English."

In [ ]:
%%time
with get_openai_callback() as callback:
    response = llm.invoke(cache_prompt)
    print(response)
    print("Total Tokens:", response.usage_metadata)
    print("*" * 100)
    print(callback)

In [ ]:
%%time
with get_openai_callback() as callback:
    response2 = llm2.invoke(prompt)
    print(response2)
    print("Total Tokens:", response.usage_metadata)
    print("*" * 100)
    print(callback)

## Cache - Redis

실습은 진행하지 않으나, 이렇게도 쓸 수 있다는걸 보여드리고
실제로는 많이 쓰는 방법 임을 말씀드립니다. (비용 절약 측면)

In [ ]:
# %pip install -U langchain-redis
# from langchain_redis.cache import RedisCache, RedisSemanticCache
# from langchain_openai import OpenAIEmbeddings, AzureOpenAIEmbeddings

# set_llm_cache(
#     RedisCache(
#         redis_client=Redis(
#             host="{YOUR_REDIS_HOST}",
#             port=6379,
#             password="{YOUR_REDIS_PASSWORD}",
#         )
#     )
# )

# Redis Semantic Cache
## 내부적으로 Redis = 캐시 + 벡터스토어 형태로 혼합하여 사용
# set_llm_cache(RedisSemanticCache(redis_url=redis_url, embeddings=AzureOpenAIEmbeddings()))

# redis_prompt1 = "What is top 10 famous street foods in Seoul Korea in 200 characters"
# redis_prompt2 = "What is top 5 famous street foods in Seoul Korea in 200 characters"

# with get_openai_callback() as callback:
#     response = llm.invoke(redis_prompt1)
#     print(response.pretty_print())
#     print("Total Tokens:", callback.total_tokens)

# with get_openai_callback() as callback:
#     llm.invoke(prompt)
#     response = llm.invoke(redis_prompt2)
#     print(response.pretty_print())
#     print("Total Tokens:", callback.total_tokens)

## LangChain Expression Language (LCEL)

### LCEL 관련 문서
#### LCEL CheatSheet: https://python.langchain.com/docs/how_to/lcel_cheatsheet/
#### Runnable API Docs: https://python.langchain.com/api_reference/core/runnables.html